In [ ]:
!pip install yfinance

import pandas as pd
import yfinance as yf
import json
from datetime import datetime
from IPython.display import clear_output 
import numpy as np
import pandas as pd
import pytz
from time import gmtime, strftime

clear_output()

In [ ]:
# example uses Apple's stock
aaplDf = yf.download('AAPL')
aaplDf

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099722,469033600
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094519,175884800
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087582,105728000
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089749,86441600
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092351,73449600
...,...,...,...,...,...,...
2023-04-11,162.350006,162.360001,160.509995,160.800003,160.800003,47644200
2023-04-12,161.220001,162.059998,159.779999,160.100006,160.100006,50133100
2023-04-13,161.630005,165.800003,161.419998,165.559998,165.559998,68445600


In [ ]:
# convert to JSON
#aaplJSON = aaplDf.to_json('aapl.json', orient='records', lines=True)
aaplDf['date'] = aaplDf.index
aaplDf['date'] = pd.to_datetime(aaplDf['date'],format='%Y-%m-%d %00:00:00.00 %Z' ).astype(str)
aaplDf.head()

,Open,High,Low,Close,Adj Close,Volume,date
Date,,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099722,469033600,1980-12-12
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094519,175884800,1980-12-15
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087582,105728000,1980-12-16
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089749,86441600,1980-12-17
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092351,73449600,1980-12-18


In [ ]:
import ast
aaplJSON  = aaplDf.to_json(orient="records")
aaplJSON  = ast.literal_eval(aaplJSON)
#aaplJSON  = aaplJSON[0:3]
aaplJSON 

[{'Open': 0.1283479929,
  'High': 0.1289059967,
  'Low': 0.1283479929,
  'Close': 0.1283479929,
  'Adj Close': 0.099721536,
  'Volume': 469033600,
  'date': '1980-12-12'},
 {'Open': 0.1222100034,
  'High': 0.1222100034,
  'Low': 0.1216519997,
  'Close': 0.1216519997,
  'Adj Close': 0.0945190042,
  'Volume': 175884800,
  'date': '1980-12-15'},
 {'Open': 0.1132809967,
  'High': 0.1132809967,
  'Low': 0.1127230003,
  'Close': 0.1127230003,
  'Adj Close': 0.0875815228,
  'Volume': 105728000,
  'date': '1980-12-16'},
 {'Open': 0.1155129969,
  'High': 0.1160710007,
  'Low': 0.1155129969,
  'Close': 0.1155129969,
  'Adj Close': 0.0897492543,
  'Volume': 86441600,
  'date': '1980-12-17'},
 {'Open': 0.1188620031,
  'High': 0.1194199994,
  'Low': 0.1188620031,
  'Close': 0.1188620031,
  'Adj Close': 0.0923513174,
  'Volume': 73449600,
  'date': '1980-12-18'},
 {'Open': 0.1261159927,
  'High': 0.1266739964,
  'Low': 0.1261159927,
  'Close': 0.1261159927,
  'Adj Close': 0.0979873389,
  'Volume': 4

In [ ]:
now= datetime.now()
date_time = now.strftime("%Y-%m-%d %H:%M:%S.%f")
tzs = 'GMT+11'
tza = 'GMT-5'

jsonfile_aapl = {"data_source":"yahoo_finance", 
                  "dataset_type": "Daily stock price", 
                  "dataset_id": "http://bucket-name.s3-website-Region.amazonaws.com", 
                 "time_object":{"timestamp":date_time, "timezone":tzs},
                  "events":[]}

for i in range(len(aaplJSON)):
  datetime_object = pd.to_datetime(aaplJSON[i]['date'], format='%Y-%m-%d %H:%M:%S.%f' )
  datetime_object= datetime_object.strftime('%Y-%m-%d %H:%M:%S.%f')
  jsonfile_aapl['events'].append({"time_object":{"timestamp":datetime_object, "duration":1, "duration_unit":"day","timezone":tzs},
                                  "event_type":'stock ohlc',   
                                  "attribute":aaplJSON[i] })

# write JSON data to file
with open("jsonfile_aapl.json", "w") as write_file:
    json.dump(jsonfile_aapl, write_file, indent=4)
    
jsonfile_aapl

{'data_source': 'yahoo_finance',
 'dataset_type': 'Daily stock price',
 'dataset_id': 'http://bucket-name.s3-website-Region.amazonaws.com',
 'time_object': {'timestamp': '2023-04-18 13:49:37.288726',
  'timezone': 'GMT+11'},
 'events': [{'time_object': {'timestamp': '1980-12-12 00:00:00.000000',
    'duration': 1,
    'duration_unit': 'day',
    'timezone': 'GMT+11'},
   'event_type': 'stock ohlc',
   'attribute': {'Open': 0.1283479929,
    'High': 0.1289059967,
    'Low': 0.1283479929,
    'Close': 0.1283479929,
    'Adj Close': 0.099721536,
    'Volume': 469033600,
    'date': '1980-12-12'}},
  {'time_object': {'timestamp': '1980-12-15 00:00:00.000000',
    'duration': 1,
    'duration_unit': 'day',
    'timezone': 'GMT+11'},
   'event_type': 'stock ohlc',
   'attribute': {'Open': 0.1222100034,
    'High': 0.1222100034,
    'Low': 0.1216519997,
    'Close': 0.1216519997,
    'Adj Close': 0.0945190042,
    'Volume': 175884800,
    'date': '1980-12-15'}},
  {'time_object': {'timestamp'